In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1.介绍**    
**2.数据获取整合**  
**3.数据预处理**  
&nbsp;&nbsp;&nbsp;&nbsp;3.1数据清洗  
&nbsp;&nbsp;&nbsp;&nbsp;3.2缺失值处理  
**4.变量筛选**  
**5.模型开发**  
&nbsp;&nbsp;&nbsp;&nbsp;5.1变量分段  
&nbsp;&nbsp;&nbsp;&nbsp;5.2WOE变换  
&nbsp;&nbsp;&nbsp;&nbsp;5.3逻辑回归估算  
**6.模型评估**  
**7.信用评分**  
**8.建立评分系统**  



In [ ]:
# Basic packages
import numpy as np
import pandas as pd
 
# Viz
import matplotlib.pyplot as plt
import seaborn as sns

from  sklearn.ensemble import RandomForestClassifier
from  sklearn.ensemble import RandomForestRegressor
 
# settings
import warnings
warnings.filterwarnings("ignore")
 
#读取数据
data = pd.read_csv(r"../input/GiveMeSomeCredit/cs-training.csv",encoding='gb18030')
#数据集确实和分布情况
data.describe().to_csv(r'DataDescribe.csv')
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
# 用随机森林对缺失值预测填充函数
def set_missing(df):
    # 把已有的数值型特征取出来
    process_df = df.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11]]
    # 分成已知该特征和未知该特征两部分
    known = process_df[process_df.MonthlyIncome.notnull()].values
    unknown = process_df[process_df.MonthlyIncome.isnull()].values
    # X为特征属性值
    X = known[:, [1,2,3,4,5,7,8,9,10]]
    # y为结果标签值
    y = known[:,[6]]
    # fit到RandomForestRegressor之中
    rfr = RandomForestRegressor(random_state=0, 
    n_estimators=200,max_depth=3,n_jobs=-1)
    rfr.fit(X,y)
    # 用得到的模型进行未知特征值预测
    predicted = rfr.predict(unknown[:,[1,2,3,4,5,7,8,9,10]]).round(0)
    print(predicted)
    # 用得到的预测结果填补原缺失数据
    print(len(predicted))
    df.loc[(df.MonthlyIncome.isnull()), 'MonthlyIncome'] = predicted
    return df

In [ ]:
data=set_missing(data)#用随机森林填补比较多的缺失值
data=data.dropna()#删除比较少的缺失值
data=data.drop_duplicates()#删除重复项
data.to_csv(r'MissingData.csv',index=False)
data.shape

In [ ]:
# 年龄等于0的异常值进行剔除
data = data[data['age']> 0]

In [ ]:
#直方图查看全部变量数据分布情况
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
datatemp2=data["DebtRatio"]
datatemp2.plot(kind='box',title='DebtRatio Distribution',sym='b+');

In [ ]:
print(data[data['DebtRatio'] > 8000].count()) # 看了看需要提出多少异常值

In [ ]:
#只有几百个样本，对于15万数据来说不影响总体数据结构，所以剔除
data = data[data['DebtRatio'] < 8000]

In [ ]:
datatemp3=data["MonthlyIncome"]
datatemp3.plot(kind='box',title='MonthlyIncome Distribution',sym='b+');

In [ ]:
print(data[data['MonthlyIncome'] > 50000].count())

In [ ]:
#剔除异常值
data = data[data['MonthlyIncome'] < 50000]

In [ ]:
datatemp4=data["NumberOfDependents"]
datatemp4.plot(kind='box',title='NumberOfDependents Distribution',sym='b+');

In [ ]:
print(data[data['NumberOfDependents'] > 10].count()) 

In [ ]:
data = data[data['NumberOfDependents'] < 10]

In [ ]:
datatemp5=data["NumberOfTime30-59DaysPastDueNotWorse"]
datatemp5.plot(kind='box',title='NumberOfTime30-59DaysPastDueNotWorse Distribution',sym='b+');

In [ ]:
print(data[data['NumberOfTime30-59DaysPastDueNotWorse'] > 20].count())

In [ ]:
#剔除异常值
data = data[data['NumberOfTime30-59DaysPastDueNotWorse'] < 20]

In [ ]:
datatemp6=data["NumberOfTime60-89DaysPastDueNotWorse"]
datatemp6.plot(kind='box',title='NumberOfTime60-89DaysPastDueNotWorse Distribution',sym='b+');

In [ ]:
datatemp7=data["NumberOfTimes90DaysLate"]
datatemp7.plot(kind='box',title='NumberOfTimes90DaysLate Distribution',sym='b+');

In [ ]:
datatemp8=data["NumberRealEstateLoansOrLines"]
datatemp8.plot(kind='box',title='NumberRealEstateLoansOrLines Distribution',sym='b+');

In [ ]:
print(data[data['NumberRealEstateLoansOrLines'] > 30].count())

In [ ]:
#剔除异常值
data = data[data['NumberRealEstateLoansOrLines'] < 30]

In [ ]:
datatemp9=data["RevolvingUtilizationOfUnsecuredLines"]
datatemp9.plot(kind='box',title='RevolvingUtilizationOfUnsecuredLines Distribution',sym='b+');

In [ ]:
print(data[data['RevolvingUtilizationOfUnsecuredLines'] > 6].count()) 

In [ ]:
#剔除异常值
data = data[data['RevolvingUtilizationOfUnsecuredLines'] < 6]

In [ ]:
data=data.drop(["Unnamed: 0"],axis=1)
data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.hist(data.MonthlyIncome,bins=70,color="red",alpha=0.7,rwidth=0.9)
plt.title("MonthlyIncome distribution")
plt.ylabel('# of MonthlyIncome', fontsize=12)
plt.xlabel('MonthlyIncome', fontsize=12)
plt.show()


In [ ]:
plt.figure(figsize=(15,5))
plt.hist(data.age,bins=50,color="green",alpha=0.7,rwidth=0.9)
plt.title("age distribution")
plt.ylabel('# of age', fontsize=12)
plt.xlabel('age', fontsize=12)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
Y = data['SeriousDlqin2yrs']
X = data.iloc[:, 1:]
#测试集占比30%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
train = pd.concat([Y_train, X_train], axis=1)
test = pd.concat([Y_test, X_test], axis=1)
 
train.to_csv(r'TrainData.csv',index=False)
test.to_csv(r'TestData.csv',index=False)

In [ ]:
# 定义自动分箱函数
from scipy import stats
def mono_bin(Y, X, n = 20):
    r = 0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates="drop")}) 
        # 后面报错You can drop duplicate edges by setting the 'duplicates' kwarg，所以回到这里补充duplicates参数
        # pandas中使用qcut()，边界易出现重复值，如果为了删除重复值设置 duplicates=‘drop’，则易出现于分片个数少于指定个数的问题
        # 经尝试，设置duplicates参数为“drop”可行，而不能设置为“raise”。
        d2 = d1.groupby('Bucket', as_index = True)
        r, p = stats.spearmanr(d2.X.mean(), d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min'] = d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d4 = (d3.sort_values(by='min')).reset_index(drop=True)
    print("=" * 60)
    woe=list(d4['woe'].round(3))
    print(d4)
    return d4

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data['DebtRatio'])

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data.RevolvingUtilizationOfUnsecuredLines)

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data.age)

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data.MonthlyIncome)

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data['NumberOfTime30-59DaysPastDueNotWorse'])

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data['NumberOfTime60-89DaysPastDueNotWorse'])

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data['NumberOfTimes90DaysLate'])

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data.NumberRealEstateLoansOrLines)

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data.NumberOfDependents)

In [ ]:
mono_bin(data.SeriousDlqin2yrs,data.NumberOfOpenCreditLinesAndLoans)

In [ ]:
# 连续变量离散化
pinf = float('inf')#正无穷大
ninf = float('-inf')#负无穷大
 
cutx6 = [ninf, 1, 2, 3, 5, pinf]
cutx7 = [ninf, 0, 1, 3, 5, pinf]
cutx8 = [ninf, 0,1,2, 3, pinf]
cutx9 = [ninf, 0, 1, 3, pinf]

In [ ]:
# 数据集各变量的相关性。
# 相关性图我们通过Python里面的seaborn包，调用heatmap()绘图函数进行绘制，实现代码如下：
 
corr = data.corr()#计算各变量的相关性系数
xticks = ['x0','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']#x轴标签
yticks = list(corr.index)#y轴标签
fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)
sns.heatmap(corr, annot=True, cmap='BrBG', ax=ax1, annot_kws={'size': 8, 'weight': 'bold',})#绘制相关性系数热力图
ax1.set_xticklabels(xticks, rotation=0, fontsize=10)
ax1.set_yticklabels(yticks, rotation=0, fontsize=10)
plt.show()


In [ ]:
# IV的实现放在mono_bin()函数里面，代码实现如下：
 
# 定义自动分箱函数
def mono_bin(Y, X, n = 20):
    r = 0
    good=Y.sum()
    bad=Y.count()-good
    while np.abs(r) < 1:
        d1 = pd.DataFrame({"X": X, "Y": Y, "Bucket": pd.qcut(X, n,duplicates="drop")})
        d2 = d1.groupby('Bucket', as_index = True)
        r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
        n = n - 1
    d3 = pd.DataFrame(d2.X.min(), columns = ['min'])
    d3['min']=d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe']=np.log((d3['rate']/(1-d3['rate']))/(good/bad))
    d3['goodattribute']=d3['sum']/good
    d3['badattribute']=(d3['total']-d3['sum'])/bad
    iv=((d3['goodattribute']-d3['badattribute'])*d3['woe']).sum()
    d4 = (d3.sort_values(by = 'min')).reset_index(drop=True)
    print("=" * 60)
    print(d4)
    cut=[]
    cut.append(float('-inf'))
    for i in range(1,n+1):
        qua=X.quantile(i/(n+1))
        cut.append(round(qua,4))
    cut.append(float('inf'))
    woe=list(d4['woe'].round(3))
    return d4,iv,cut,woe

In [ ]:
#自定义分箱函数
def self_bin(Y,X,cat):
    good=Y.sum()
    bad=Y.count()-good
    d1=pd.DataFrame({'X':X,'Y':Y,'Bucket':pd.cut(X,cat)})
    d2=d1.groupby('Bucket', as_index = True)
    d3 = pd.DataFrame(d2.X.min(), columns=['min'])
    d3['min'] = d2.min().X
    d3['max'] = d2.max().X
    d3['sum'] = d2.sum().Y
    d3['total'] = d2.count().Y
    d3['rate'] = d2.mean().Y
    d3['woe'] = np.log((d3['rate'] / (1 - d3['rate'])) / (good / bad))
    d3['goodattribute'] = d3['sum'] / good
    d3['badattribute'] = (d3['total'] - d3['sum']) / bad
    iv = ((d3['goodattribute'] - d3['badattribute']) * d3['woe']).sum()
    d4 = (d3.sort_values(by='min'))
    print("=" * 60)
    print(d4)
    woe = list(d4['woe'].round(3))
    return d4, iv,woe

In [ ]:
dfx1, ivx1,cutx1,woex1 = mono_bin(data.SeriousDlqin2yrs, data.RevolvingUtilizationOfUnsecuredLines,n=10)
dfx2, ivx2,cutx2,woex2 = mono_bin(data.SeriousDlqin2yrs, data.age, n=10)
dfx4, ivx4,cutx4,woex4 =mono_bin(data.SeriousDlqin2yrs, data.DebtRatio, n=20)
dfx5, ivx5,cutx5,woex5 =mono_bin(data.SeriousDlqin2yrs, data.MonthlyIncome, n=10)
 
# 连续变量离散化
cutx3 = [ninf, 0, 1, 3, 5, pinf]
cutx6 = [ninf, 1, 2, 3, 5, pinf]
cutx7 = [ninf, 0, 1, 3, 5, pinf]
cutx8 = [ninf, 0,1,2, 3, pinf]
cutx9 = [ninf, 0, 1, 3, pinf]
cutx10 = [ninf, 0, 1, 2, 3, 5, pinf]
dfx3, ivx3,woex3 = self_bin(data.SeriousDlqin2yrs, data['NumberOfTime30-59DaysPastDueNotWorse'], cutx3)
dfx6, ivx6,woex6= self_bin(data.SeriousDlqin2yrs, data['NumberOfOpenCreditLinesAndLoans'], cutx6)
dfx7, ivx7,woex7 = self_bin(data.SeriousDlqin2yrs, data['NumberOfTimes90DaysLate'], cutx7)
dfx8, ivx8,woex8 = self_bin(data.SeriousDlqin2yrs, data['NumberRealEstateLoansOrLines'], cutx8)
dfx9, ivx9,woex9 = self_bin(data.SeriousDlqin2yrs, data['NumberOfTime60-89DaysPastDueNotWorse'], cutx9)
dfx10, ivx10,woex10 = self_bin(data.SeriousDlqin2yrs, data['NumberOfDependents'], cutx10)

In [ ]:
# 生成IV图：
ivlist=[ivx1,ivx2,ivx3,ivx4,ivx5,ivx6,ivx7,ivx8,ivx9,ivx10]#各变量IV
index=['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']#x轴的标签
fig1 = plt.figure(1)
ax1 = fig1.add_subplot(1, 1, 1)
x = np.arange(len(index))+1
ax1.bar(x, ivlist, width=0.4,color='g',alpha=0.7)#生成柱状图
ax1.set_xticks(x)
ax1.set_xticklabels(index, rotation=0, fontsize=12)
ax1.set_ylabel('IV(Information Value)', fontsize=14)
#在柱状图上添加数字标签
for a, b in zip(x, ivlist):
    plt.text(a, b + 0.01, '%.4f' % b, ha='center', va='bottom', fontsize=10)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn import metrics
import statsmodels.api as sm
import math
 
def trans_woe(var,var_name,x_woe,x_cut):
    woe_name = var_name + '_woe'
    for i in range(len(x_woe)):
        if i == 0:
            var.loc[(var[var_name]<=x_cut[i+1]),woe_name] = x_woe[i]
        elif (i>0) and (i<= len(x_woe)-2):
            var.loc[((var[var_name]>x_cut[i])&(var[var_name]<=x_cut[i+1])),woe_name] = x_woe[i]
        else:
            var.loc[(var[var_name]>x_cut[len(x_woe)-1]),woe_name] = x_woe[len(x_woe)-1]
    return var
    
x1_name = 'RevolvingUtilizationOfUnsecuredLines'
x2_name = 'age'
x3_name = 'NumberOfTime30-59DaysPastDueNotWorse'
x7_name = 'NumberOfTimes90DaysLate'
x9_name = 'NumberOfTime60-89DaysPastDueNotWorse'
 
X_train = trans_woe(X_train,x1_name,woex1,cutx1)
X_train = trans_woe(X_train,x2_name,woex2,cutx2)
X_train = trans_woe(X_train,x3_name,woex3,cutx3)
X_train = trans_woe(X_train,x7_name,woex7,cutx7)
X_train = trans_woe(X_train,x9_name,woex9,cutx9)

In [ ]:
X_train.to_csv('WoeData.csv', index=False)
#6.2 Logistic模型建立
#导入数据
data = pd.read_csv('WoeData.csv')
#应变量
Y=train['SeriousDlqin2yrs']
# dict_Y = {'month':Y.index,'numbers':Y.values}
# df_month = pd.DataFrame(dict_Y)
#自变量，剔除对因变量影响不明显的变量
X=X_train.drop(['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate','NumberOfTime60-89DaysPastDueNotWorse','DebtRatio','MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines','NumberOfDependents'],axis=1)
X1=sm.add_constant(X)
Y1=Y.to_frame(name='SeriousDlqin2yrs')
logit=sm.Logit(Y,X1)
result=logit.fit()
print(result.summary())
X1.columns

In [ ]:
#6.3 
X_test = trans_woe(X_test,x1_name,woex1,cutx1)
X_test = trans_woe(X_test,x2_name,woex2,cutx2)
X_test = trans_woe(X_test,x3_name,woex3,cutx3)
X_test = trans_woe(X_test,x7_name,woex7,cutx7)
X_test = trans_woe(X_test,x9_name,woex9,cutx9)
 
#应变量
Y_test = test['SeriousDlqin2yrs']
#自变量，剔除对因变量影响不明显的变量，与模型变量对应
X_test = X_test.iloc[:,-5:]
#X_test =X_test.drop(['NumberOfOpenCreditLinesAndLoans','age','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate','NumberOfTime60-89DaysPastDueNotWorse','DebtRatio','MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans','NumberRealEstateLoansOrLines','NumberOfDependents'], axis=1)
X3 = sm.add_constant(X_test)
resu = result.predict(X3)#进行预测
#result.score(X3,Y_test) 
fpr, tpr, threshold = metrics.roc_curve(Y_test, resu)
rocauc = metrics.auc(fpr, tpr)#计算AUC
plt.plot(fpr, tpr, 'b', label='AUC = %0.4f' % rocauc)#生成ROC曲线
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('真正率')
plt.xlabel('假正率')
plt.show()

In [ ]:
# 取600分为基础分值，PDO为20（每高20分好坏比翻一倍），好坏比取20。
p = 20 / math.log(2)
q = 600 - 20 * math.log(20) / math.log(2)
#7.2 部分评分
#计算分数函数 
def get_score(coe,woe,factor):
    scores=[]
    for w in woe:
        score=round(coe*w*factor,0)
        scores.append(score)
    print(scores)    
    return scores
   
 
 
coe = [2.6138,0.6228,0.4894,0.5596,0.5747 ,0.4248]#在上面那个logistic的结果中来
baseScore = round(q + p * coe[0], 0)
print(baseScore)

In [ ]:
# 各项部分分数
x1 = get_score(coe[1], woex1, p)
x2 = get_score(coe[2], woex2, p)
x3 = get_score(coe[3], woex3, p)
x7 = get_score(coe[4], woex7, p)
x9 = get_score(coe[5], woex9, p)

In [ ]:
def compute_score(series,cut,score):
    list = []
    i = 0
    while i < len(series):
        value = series[i]
        j = len(cut) - 2
        m = len(cut) - 2
        while j >= 0:
            if value >= cut[j]:
                j = -1
            else:
                j -= 1
                m -= 1
        list.append(score[m])
        i += 1
    return list

In [ ]:
test1 = pd.read_csv(r'TestData.csv')
test1['BaseScore']=np.zeros(len(test1))+baseScore
test1['x1'] = compute_score(test1['RevolvingUtilizationOfUnsecuredLines'], cutx1, x1)
test1['x2'] = compute_score(test1['age'], cutx2, x2)
test1['x3'] = compute_score(test1['NumberOfTime30-59DaysPastDueNotWorse'], cutx3, x3)
test1['x7'] = compute_score(test1['NumberOfTimes90DaysLate'], cutx7, x7)
test1['x9'] = compute_score(test1['NumberOfTime60-89DaysPastDueNotWorse'], cutx9, x9)
test1['Score'] = test1['x1'] + test1['x2'] + test1['x3'] + test1['x7'] +test1['x9']  + baseScore
test1.to_csv(r'ScoreData.csv', index=False)